### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 08 декабря 2020, 08:30   
**Штраф за опоздание:** -2 балла после 08:30 08 декабря, -4 балла после 08:30 15 декабря, -6 баллов после 08:30 22 декабря, -8 баллов после 08:30 29 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0220, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс DecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier


In [279]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, features_count=10000, min_features=10,  min_samples_split=2, max_depth=5, criterion='gini'):
        """
        criterion -- критерий расщепления. необходимо релизовать три:
        Ошибка классификации, Индекс Джини, Энтропийный критерий
        max_depth -- максимальная глубина дерева
        min_samples_split -- минимальное число объектов в листе, чтобы сделать новый сплит
        """
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.num_class = -1
        # Для последнего задания
        self.features = np.zeros(features_count)
        self.criterion = criterion
        self.min_features = min_features
        # Структура, которая описывает дерево
        # Представляет словарь, где для  node_id (айдишник узла дерева) храним
        # (тип_узла, айдишник признака сплита, порог сплита) если тип NON_LEAF_TYPE
        # (тип_узла, предсказание класса, вероятность класса) если тип LEAF_TYPE
        # Подразумевается, что у каждого node_id в дереве слева 
        # узел с айди 2 * node_id + 1, а справа 2 * node_id + 2
        self.tree = dict()

    def __div_samples(self, x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
    
    def __sub_calc_loss(self, y_left, y_right, cls):
        total = 0
        if len(y_left):
            res = 0
            for x in cls:
                cur = np.count_nonzero(y_left == x) / len(y_left)
                if self.criterion == 'gini':
                    res += cur ** 2
                elif self.criterion == 'enthropy':
                    res += cur * np.log(cur)
                else:
                    res = max(res, cur)
            if self.criterion == 'enthropy':
                total += (1 - res) * len(y_left) / (len(y_left) + len(y_right))
            else:
                total -= res * len(y_left) / (len(y_left) + len(y_right))
        return total

    def calc_loss(self, y_left, y_right, cls):
        return self.__sub_calc_loss(y_left, y_right, cls) + self.__sub_calc_loss(y_right, y_left, cls)
    
    def __find_threshold(self, x, y):
        """
        Находим оптимальный признак и порог для сплита
        Здесь используемые разные impurity в зависимости от self.criterion
        """
        all_cls = set(y)
        score_min_res = 1
        index_res = 0
        value_res = 0
        for cur in x:
            for ind, val in enumerate(cur[:min(self.min_features, len(cur))]):
                x_left, x_right, y_left, y_right = self.__div_samples(x, y, ind, val)
                cur_res = self.calc_loss(y_left, y_right, all_cls)
                if cur_res < score_min_res:
                    score_min_res = cur_res
                    index_res = ind
                    value_res = val
        return index_res, value_res, score_min_res

    def __fit_node(self, x, y, ind, value, cur_score, node_id, depth):
        """
        Делаем новый узел в дереве
        Решаем, терминальный он или нет
        Если нет, то строим левый узел  с айди 2 * node_id + 1
        И правый узел с  айди 2 * node_id + 2
        """
        train_len = len(y)
        
        if depth >= self.max_depth or train_len <= self.min_samples_split:
            cur = np.argmax(np.bincount(y))
            self.tree[node_id] = [self.LEAF_TYPE, cur, np.count_nonzero(cur == y) / train_len]
            return
        
        self.tree[node_id] = [self.NON_LEAF_TYPE, ind, value]
        x_left, x_right, y_left, y_right = self.__div_samples(x, y, ind, value)
        if not len(y_left) or not len(y_right):
            cur = np.argmax(np.bincount(y))
            self.tree[node_id] = [self.LEAF_TYPE, cur, np.count_nonzero(cur == y) / train_len]
            return
        
        self.features[ind] += cur_score
        node_id_left, value_left, score_left = self.__find_threshold(x_left, y_left)
        self.features[ind] += score_left * len(y_left) / train_len
        
        node_id_right, value_right, score_right = self.__find_threshold(x_right, y_right)
        self.features[ind] += score_right * len(y_right) / train_len
    
        self.__fit_node(x_left, y_left, node_id_left, value_left, score_left, node_id * 2 + 1, depth + 1)
        self.__fit_node(x_right, y_right, node_id_right, value_right, score_right, node_id * 2 + 2, depth + 1)
    
    def fit(self, x, y):
        """
        Рекурсивно строим дерево решений
        Начинаем с корня node_id 0
        """
        ind, value, score = self.__find_threshold(x, y)
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, ind, value, score, 0, 0)

    def __predict_class(self, x, node_id):
        """
        Рекурсивно обходим дерево по всем узлам,
        пока не дойдем до терминального
        """
        #print(node_id, self.tree[node_id])
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]
        
    def predict(self, X):
        """
        Вызывает predict для всех объектов из матрицы X
        """
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)
    
    def get_feature_importance(self, features_length=100, most_important_count=10):
        """
        Возвращает важность признаков
        """
        return np.argpartition(self.features[:features_length], 
                               -most_important_count)[-most_important_count:]

In [325]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, min_features=7, features_count=100)
clf = DecisionTreeClassifier(min_samples_split=2)

In [326]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [327]:
my_clf.fit(X_train, y_train)
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [328]:
print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))
print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.8333333333333334
0.8333333333333334


## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine. 
Для этого используем numpy.

In [336]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 4.58 ms, total: 4.58 ms
Wall time: 4.81 ms


DecisionTreeClassifier()

In [339]:
%time my_clf.fit(X_train, y_train)

CPU times: user 32.7 ms, sys: 6.93 ms, total: 39.6 ms
Wall time: 37.9 ms


## Боевое применение (3 балла)

На практике Вы познакомились с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match). 

Пример работы с датасетом можете найти в практике пункт 2
https://github.com/VVVikulin/ml1.sphere/blob/master/2019-09/lecture_06/pract-trees.ipynb

Данные и описания колонок лежат тут
https://cloud.mail.ru/public/8nHV/p6J7wY1y1/speed-dating-experiment/

Скачайте датасет, обработайте данные, как показано на семинаре или своим собственным способом. Обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. 


Разбейте датасет на трейн и валидацию. Подберите на валидации оптимальный критерий  информативности. 
Постройте графики зависимости точности на валидации от глубины дерева, от минимального числа объектов для сплита. 
Какой максимальной точности удалось достигнуть?

## Находим самые важные признаки (2 балла)



По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве. 

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший. 
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева. 

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ

